In [ ]:
# Estos dos comandos evitan que haya que hacer reload cada vez que se modifica un paquete
%load_ext autoreload
%autoreload 2

# Ejercicio de clasificación de texto

Naive Bayes es una técnica estadística que consiste en repetir el método anterior en problemas cuyos sucesos no son independientes, pero suponiendo independencia.
A lo largo de este trabajo desarrollarán un modelo de Naive Bayes para el problema de clasificación de artículos periodístios.En este caso podemos estimar la probabilidad de ocurrencia de cada palabra según la categoría a la que pertenece el artículo.

## Dataset


El primer paso es obtener el dataset que vamos a utilizar. El dataset a utilizar es el de TwentyNewsGroup(TNG) que está disponible en sklearn.

Se puede encontrar más información del dataset en la documentación de scikit-learn.

In [84]:
#Loading the data set - training data.
from sklearn.datasets import fetch_20newsgroups
from os.path import isfile
import pickle

TT_FILE = 'twenty-train-nometadata.p'
if isfile(TT_FILE):
    twenty_train = pickle.load(open(TT_FILE, 'rb'))
else:
    twenty_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), shuffle=True)
    pickle.dump(twenty_train, open(TT_FILE, 'wb'))

El siguiente paso es analizar el contenido del dataset, como por ejemplo la cantidad de artículos, la cantidad de clases, etc.

Preguntas:

1) ¿Cuántos articulos tiene el dataset?

In [85]:
len(twenty_train.data)

11314

2) ¿Cuántas clases tiene el dataset?

In [86]:
len(twenty_train.target_names)

20

3) ¿Es un dataset balanceado?

In [87]:
import numpy as np
_, counts = np.unique(twenty_train.target, return_counts=True)
if len(set(counts)) == 1:
    print('El dataset está balanceado')
else:    
    print('El dataset no está balanceado')

El dataset no está balanceado


4) ¿Cuál es la probabilidad a priori de la clase 5? A que corresponde esta clase?

In [88]:
priori5 = counts[5]/sum(counts)
print(f'La clase 5 ({twenty_train.target_names[5]}) tiene probabilidad a priori {priori5:.2}')

La clase 5 (comp.windows.x) tiene probabilidad a priori 0.052


5) ¿Cuál es la clase con mayor probabilidad a priori?

In [ ]:
max_class = np.argmax(counts)
print(f'La clase {max_class} ({twenty_train.target_names[max_class]}) tiene la máxima probabilidad a priori, {counts[max_class]/sum(counts):.2}')

## Preprocesamiento

Para facilitar la comprensión de los algoritmos de preprocesamiento, se aplican primero a un solo artículo.


Mas info en:
http://text-processing.com/demo/stem/

In [ ]:
article = twenty_train.data[0]
article

- **Tokenization (nltk):**

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

tok = word_tokenize(article)
tok


- **Lemmatization (nltk):**


In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lem=[lemmatizer.lemmatize(x,pos='v') for x in tok]
lem

- **Stop Words (nltk):**


In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = [x for x in lem if x not in stopwords.words('english')]
stop

- **Stemming (nltk):**


In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stem = [stemmer.stem(x) for x in stop]
stem

- **Filtrado de palabras:**


In [ ]:
alpha = [x for x in stem if x.isalpha()]
alpha

### Preprocesamiento completo

Utilizar o no cada uno de los métodos vistos es una decisión que dependerá del caso particular de aplicación. Para este ejercicio vamos a considerar las siguientes combinaciones:

- Tokenización
- Tokenización, Lematización, Stemming.
- Tokenización, Stop Words.
- Tokenización, Lematización, Stop Words, Stemming.
- Tokenización, Lematización, Stop Words, Stemming, Filtrado.

In [89]:
def filter_article(article, filts):
    filts = filts.split()

    if 'lem' in filts:
        article = [lemmatizer.lemmatize(x,pos='v') for x in article]

    if 'stop' in filts:
        article = [x for x in article if x not in stopwords.words('english')]

    if 'stem' in filts:
        article = [stemmer.stem(x) for x in article]

    if 'filt' in filts:
        article = [x for x in article if x.isalpha()]
    
    return article


def filter_articles(name, articles, filts):
    
    filename = f'{name}-{preproc}.p'
    if os.path.isfile(filename):
        with open (filename, 'rb') as fp:
            filtered_articles = pickle.load(fp)
    
    else:
        filtered_articles = []
        for data in articles:
            tok = word_tokenize(data)
            filtered_articles.append(filter_article(tok, filts))
        
        with open(filename, 'wb') as fp:
            pickle.dump(filtered_articles, fp)
            
    return filtered_articles
    
        

In [90]:
ans_fmt = """Preprocesamiento: {preproc}
Longitud del vocabulario: {vocab_len}
"""
preprocessing = ['tok', 'tok lem stem', 'tok stop', 'tok lem stop stem', 'tok lem stop stem filt']

    
for preproc in preprocessing:
    filtered_articles = filter_articles('train-nometadata', twenty_train.data, preproc)
    vocab = set([word for article in filtered_articles for word in article])
    print(ans_fmt.format(preproc=preproc, vocab_len=len(vocab)))

Preprocesamiento: tok
Longitud del vocabulario: 161698

Preprocesamiento: tok lem stem
Longitud del vocabulario: 126903

Preprocesamiento: tok stop
Longitud del vocabulario: 161533

Preprocesamiento: tok lem stop stem
Longitud del vocabulario: 126872

Preprocesamiento: tok lem stop stem filt
Longitud del vocabulario: 45315



Preguntas

- Cómo cambia el tamaño del vocabulario al agregar Lematización y Stemming?
- Cómo cambia el tamaño del vocabulario al Stop Words?
- Analice muy brevemente ventajas y desventajas del tamaño del dataset en cada caso.

## Vectorización de texto

- **Obtención del vocabulario y obtención de la probabilidad**

Como se vió en clase, los vectorizadores cuentan con dos parámetros de ajuste.

- max_df: le asignamos una maxima frecuencia de aparición, eliminando las palabras comunes que no aportan información.

- min_df: le asignamos la minima cantidad de veces que tiene que aparecer una palabra.


In [91]:
joined_file_fmt = '{name}-joined-{filts}.p'
for preproc in preprocessing:
    filename = joined_file_fmt.format(name='train-nometadata', filts=preproc)

    if not os.path.isfile(filename):
        with open(f'train-nometadata-{preproc}.p', 'rb') as fp:
            articles = pickle.load(fp)
            for i in range(len(articles)):
                articles[i] = ' '.join(articles[i])
            
            with open(filename, 'wb') as fp_joined:
                pickle.dump(articles, fp_joined)

In [57]:
def get_filtered_joined_articles(name, filts):
    filename = joined_file_fmt.format(name=name, filts=filts)
    with open(filename, 'rb') as fp:
            articles = pickle.load(fp)
    return articles

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
import pandas as pd

OUT_FILE = 'out-nometadata.csv'

vectorizers = [CountVectorizer, TfidfVectorizer]
classifiers = [MultinomialNB, BernoulliNB]
max_dfs = [0.50, 0.75, 0.80, 0.90, 0.95]
min_dfs = [int((round(len(twenty_train.data)*n/100))) for n in [0.01, 0.1, 0.5, 1]]

results = []

import pandas as pd


total_its = len(vectorizers)*len(classifiers)*len(min_dfs)*len(max_dfs)*len(preprocessing)
i = 1

if os.path.isfile(OUT_FILE):
    results = pd.read_csv(OUT_FILE)
else:
    results = []

    for filts in preprocessing:
        data = get_filtered_joined_articles(name='train-nometadata', filts=filts) 
        
        for max_df in max_dfs:
            for min_df in min_dfs:
                
                for Vectorizer in vectorizers:
                    count_vect = Vectorizer(max_df=max_df, min_df=min_df)
                    raw_data = count_vect.fit_transform(data) 

                    for Classifier in classifiers:
                        print(f'Computing: F={filts}, M={max_df}, m={min_df}, V={Vectorizer.__name__}, C={Classifier.__name__} ({i}/{total_its})...')
                        i += 1
                        
                        clf = Classifier()
                        clf.fit(raw_data, twenty_train.target)
                        score = clf.score(raw_data, twenty_train.target)

                        results.append({
                            'max_df': max_df,
                            'min_df': min_df,
                            'filts': filts,
                            'vectorizer': Vectorizer.__name__,
                            'model': Classifier.__name__,
                            'score': score
                        })
                        
                        print(f'Score: {score:.5%}')
                        print(50 * '-')
    results = pd.DataFrame(results)
    results.to_csv(OUT_FILE, index=False)

In [93]:
best = results['score'].max()
best_filt = results['score']==best
best_hypers = results[best_filt]
best_hypers

,max_df,min_df,filts,vectorizer,model,score
320,0.5,1,tok lem stop stem filt,CountVectorizer,MultinomialNB,0.955895


In [81]:
results[results.model=='BernoulliNB'].sort_values(by=['score'], ascending=False).head(60)

,max_df,min_df,filts,vectorizer,model,score
165,0.50,11,tok stop,CountVectorizer,BernoulliNB,0.858494
167,0.50,11,tok stop,TfidfVectorizer,BernoulliNB,0.858494
215,0.90,11,tok stop,TfidfVectorizer,BernoulliNB,0.854870
213,0.90,11,tok stop,CountVectorizer,BernoulliNB,0.854870
231,0.95,11,tok stop,TfidfVectorizer,BernoulliNB,0.854870
229,0.95,11,tok stop,CountVectorizer,BernoulliNB,0.854870
181,0.75,11,tok stop,CountVectorizer,BernoulliNB,0.854870
183,0.75,11,tok stop,TfidfVectorizer,BernoulliNB,0.854870
197,0.80,11,tok stop,CountVectorizer,BernoulliNB,0.854870
199,0.80,11,tok stop,TfidfVectorizer,BernoulliNB,0.854870


## Entrenamiento del modelo

Primero deben separar correctamente el dataset para hacer validación del modelo.

Y luego deben entrenar el modelo de NaiveBayes con el dataset de train.

Deben utilizar un modelo de NaiveBayes Multinomial y de Bernoulli. Ambos modelos estan disponibles en sklearn.

In [ ]:


#Completar para cada caso según corresponda
clf = MultinomialNB()
# clf = BernoulliNB()
clf.fit(raw_data, twenty_train.target)


Finalmente comprobar el accuracy en train.

In [ ]:
import numpy as np
porc=clf.score(raw_data, twenty_train.target)
print("El porcentaje de artículos clasificados correctamente es: {:.2f}%".format(porc))

Preguntas

- Con que combinación de preprocesamiento obtuvo los mejores resultados? Explique por qué cree que fue así.

- Con que modelo obtuvo los mejores resultados? Explique por qué cree que fue así.

## Performance de los modelos

En el caso anterior, para medir la cantidad de artículos clasiicados correctamente se utilizó el mismo subconjunto del dataset que se utilizó para entrenar.

Esta medida no es una medida del todo útil, ya que lo que interesa de un clasificador es su capacidad de clasificación de datos que no fueron utilizados para entrenar. Es por eso que se pide, para el clasificador entrenado con el subconjunto de training, cual es el porcentaje de artículos del subconjunto de testing clasificados correctamente. Comparar con el porcentaje anterior y explicar las diferencias.

Finalmente deben observar las diferencias y extraer conclusiones en base al accuracy obtenido, el preprocesamiento y vectorización utilizado y el modelo, para cada combinación de posibilidades.

In [ ]:
# Solución

In [ ]:
import numpy as np
porc=sum(np.array(clf.predict(X_test.toarray()))==np.array(Y_test))/len(Y_test)*100
print("El porcentaje de artículos clasificados correctamente es: {:.2f}%".format(porc))

Preguntas

- El accuracy en el dataset de test es mayor o menor que en train? Explique por qué.